In [1]:
from werkzeug.wrappers import Request, Response
#from flask import Flask, render_template
from jinja2 import Template

#from datetime import date
from datetime import datetime, timedelta

import collections
from amigocloud import AmigoCloud

from datetime import datetime

from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm
import requests

C:\Users\Bismar\.conda\envs\utea_reportes\lib\importlib\_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
C:\Users\Bismar\.conda\envs\utea_reportes\lib\importlib\_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
C:\Users\Bismar\.conda\envs\utea_reportes\lib\importlib\_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
C:\Users\Bismar\.conda\envs\utea_reportes\lib\importlib\_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)


In [2]:
acceso = open('../../api_key/api.key','r')
api_token = acceso.readlines()[0]
acceso.close()

amigocloud = AmigoCloud(token=api_token)
amigocloud

In [3]:
semana = 13

In [4]:
query_semana = {'query': 'select id, canhero from dataset_315614 where date_part(\'week\', fecha_de_registro)={sem} order by id'.format(sem=semana)}
project = 'https://app.amigocloud.com/api/v1/projects/32920/sql'
select_semana = amigocloud.get(project, query_semana)
data_semana = select_semana['data']

In [5]:
data_semana

[{'id': 4, 'canhero': '10697 / PAZ REA JAIME EDUARDO '},
 {'id': 5, 'canhero': '75 / AGUILERA TARADELLES NELSON MARIANO '},
 {'id': 6, 'canhero': '75 / AGUILERA TARADELLES NELSON MARIANO '},
 {'id': 7, 'canhero': '515 / AGUILERA OLGA RIVERO VDA. DE '},
 {'id': 8, 'canhero': '8009 / ESPINOZA AGUILAR DAVID'},
 {'id': 9, 'canhero': '8009 / ESPINOZA AGUILAR DAVID'},
 {'id': 10, 'canhero': '594 / AGROPECUARIA FLAMBOYAN'},
 {'id': 11, 'canhero': '3400 / ESPINOZA AGUILAR JAIME OMAR '},
 {'id': 12, 'canhero': '574 / AYALA ESPINOZA ARTURO '},
 {'id': 13, 'canhero': '765 / AGROP. CATALA S.R.L.'},
 {'id': 14, 'canhero': '765 / AGROP. CATALA S.R.L.'},
 {'id': 15, 'canhero': '765 / AGROP. CATALA S.R.L.'},
 {'id': 16, 'canhero': '765 / AGROP. CATALA S.R.L.'},
 {'id': 17, 'canhero': '805 / AGROP. TRES HNOS DEL ORIENTE'}]

In [8]:
for id in data_semana:
    print('ID inspeccion: ', id['id'])
    id_insp = id['id']
    # DATOS DE LAS INPECCION
    query = {'query': 'select * from dataset_315614 where id={id_insp}'.format(id_insp=id_insp)}
    project = 'https://app.amigocloud.com/api/v1/projects/32920/sql'
    select = amigocloud.get(project, query)
    data = select['data'][0]

    # convertir fecha a formato dd/mm/aaaa
    data['fecha_de_registro'] = datetime.strptime(data['fecha_de_registro'][0:10], '%Y-%m-%d').strftime('%d/%m/%Y')

    # los campos booleanos tiene formacto 'num_num', esto no es admitido para convertir a objeto
    # se modifica el dict para agregar 'k_' al inicio de todas llaves
    data = {'k_'+key : value for (key, value) in data.items()}
    insp = collections.namedtuple("insp", data.keys())(*data.values())
    
    # FOTOS
    query_fotos = {'query': 'select gal.s3_filename, fot.obs from dataset_315689 fot\
                                inner join gallery_48937 gal on fot.amigo_id=gal.source_amigo_id\
                                where fot.insp_ref_fot=\'{id}\''.format(id=insp.k_amigo_id)}
    project = 'https://app.amigocloud.com/api/v1/projects/32920/sql'
    fotos = amigocloud.get(project, query_fotos)['data']
    
    # CREAR REPORTE
    doc = DocxTemplate("templates/template_insp_cosechadoras_pre.docx")
    #descargar fotos y generar lista InlineImage
    lista_fotos_inline = []
    for foto in fotos:
        response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['s3_filename'])
        file = open('fotos/' + foto['s3_filename'], "wb")
        file.write(response.content)
        file.close()
        lista_fotos_inline.append({
            'foto': docxtpl.InlineImage(doc, image_descriptor='fotos/' + foto['s3_filename'], width=Mm(50)),
            'obs': foto['obs']
        })
    context = {'insp':insp, 'fotos':lista_fotos_inline}
    #context = {'insp':insp}
    doc.render(context)
    # formato de nombre de archivo: "123_CQ_01-01-2022_NOMBRE"
    cod_nom = insp.k_canhero.split(' / ')
    file_name = cod_nom[0] + '_ECP_' + insp.k_fecha_de_registro.replace('/','-') + '_' + cod_nom[1] + '_' + str(insp.k_id)
    #doc.save(r'C:\Users\UTEA\OneDrive - Ingenio Azucarero Guabira S.A (2)\INFORMES\IMFORMES PARA REVISAR\\' + file_name + '.docx')
    doc.save(r'_' + file_name + '.docx')
    print(file_name)
print('-------------------FIN')

ID inspeccion:  4
10697_ECP_28-03-2023_PAZ REA JAIME EDUARDO _4
ID inspeccion:  5
75_ECP_28-03-2023_AGUILERA TARADELLES NELSON MARIANO _5
ID inspeccion:  6
75_ECP_28-03-2023_AGUILERA TARADELLES NELSON MARIANO _6
ID inspeccion:  7
515_ECP_29-03-2023_AGUILERA OLGA RIVERO VDA. DE _7
ID inspeccion:  8
8009_ECP_29-03-2023_ESPINOZA AGUILAR DAVID_8
ID inspeccion:  9
8009_ECP_29-03-2023_ESPINOZA AGUILAR DAVID_9
ID inspeccion:  10
594_ECP_30-03-2023_AGROPECUARIA FLAMBOYAN_10
ID inspeccion:  11
3400_ECP_30-03-2023_ESPINOZA AGUILAR JAIME OMAR _11
ID inspeccion:  12
574_ECP_30-03-2023_AYALA ESPINOZA ARTURO _12
ID inspeccion:  13
765_ECP_31-03-2023_AGROP. CATALA S.R.L._13
ID inspeccion:  14
765_ECP_31-03-2023_AGROP. CATALA S.R.L._14
ID inspeccion:  15
765_ECP_31-03-2023_AGROP. CATALA S.R.L._15
ID inspeccion:  16
765_ECP_31-03-2023_AGROP. CATALA S.R.L._16
ID inspeccion:  17
805_ECP_31-03-2023_AGROP. TRES HNOS DEL ORIENTE_17
-------------------FIN
